In [ ]:
from matplotlib.pyplot import yscale
import matplotlib.pyplot as plt
from astropy.timeseries import LombScargle
from matplotlib import cm
import numpy as np
import pandas as pd
import lightkurve as lk
from altaipony.flarelc import FlareLightCurve
from lightkurve.lightcurvefile import TessLightCurveFile
from lightkurve.lightcurve import TessLightCurve
from altaipony.ffd import FFD
import math

In [ ]:
stats = ['ID','Top_Period','Top_Period_Power','Fap_99','Is_Harmonic','Num_Flares','Is_Powerlaw']
ids = []
top_periods = []
top_period_powers = []
fap_99s = []
is_harmonic = []
num_flares = []
is_powerlaw = []

In [ ]:
%matplotlib inline

In [ ]:
def make_plot(tic_id, data_dir): 
    
    ids.append(tic_id)
    lc_df = pd.read_csv(data_dir+'/TIC '+tic_id+'.CSV')
    ls_df = lc_df[['time', 'flux', 'flux_err']]

    ls_df = ls_df.dropna()

    raw_times = np.array(ls_df['time'])
    raw_fluxes = np.array(ls_df['flux'])
    raw_flux_errs = np.array(ls_df['flux_err'])
    
    light_curve = lk.LightCurve(time=raw_times, flux=raw_fluxes, flux_err=raw_flux_errs)
    light_curve = light_curve.remove_outliers(sigma=3).normalize()

    times = np.array(light_curve.time.value)
    fluxes = np.array(light_curve.flux.value)
    flux_errs = np.array(light_curve.flux_err)
    
    # FIX to use detrended / clipped lc

    ### MAKE PLOT
    plt.style.use('C:/Users/60002/Documents/GitHub/sunnyhills/other/aesthetics/science.mplstyle') # set style...should work on all our computers because relative

    # try with "root path" set to "./other/aesthetics/science.mplstyle"

    mosaic = """
        AAB
        CCD
        EEF
        """

    fig = plt.figure(constrained_layout=True, figsize=(12,6))
    ax_dict = fig.subplot_mosaic(mosaic)

    # A is raw light curve
    ax_dict['A'].scatter(raw_times, raw_fluxes, s=3)
    ax_dict['A'].set(xlabel='Time (d)', ylabel='Raw Normalized Flux')

    # to axis one of the subplots, do it like this: ax_dict['A'] (e.g. for subplot A)

    # lomb-periodogram 
    # max frequency = observation window / 3
    # min frequency = ???

    # ls won't work when nans in array 
    
    window_length = np.max(times)-np.min(times)
    periodogram = LombScargle(times, fluxes, flux_errs)

    false_alarm_levels = [0.01]
    faps = periodogram.false_alarm_level(false_alarm_levels)

    frequencies, powers = periodogram.autopower(minimum_frequency=3/window_length, maximum_frequency=10)
    # max freq and min freq 
    periods = 1/frequencies 

    
    lc = FlareLightCurve(raw_times,raw_fluxes,raw_flux_errs,mission='TESS')

    detrended = lc.detrend("savgol")
    detrended = detrended.find_flares()
    simple_ffd= FFD(f=detrended.flares)

    simple_ffd.tot_obs_time = 20
    ed, freq,count= simple_ffd.ed_and_freq()

    ax_dict['B'].scatter(ed, freq)
    ax_dict['B'].set(xscale="log",yscale="log")
    ax_dict['B'].set(xLabel="ED [s]",yLabel="cumulative num of flares")

    try:
        simple_ffd.fit_powerlaw("mmle")
        simple_ffd.plot_powerlaw(ax_dict['B'], c="r", label=fr'$\alpha=$-{simple_ffd.alpha:.1f}')
        is_powerlaw.append(simple_ffd.is_powerlaw(sig_level=.05))
    except ValueError:
        is_powerlaw.append(math.nan)
        

    num_flares.append(detrended.flares.shape[0])
    try:
        for start,stop in zip(detrended.flares['tstart'],detrended.flares['tstop']):
            ax_dict['C'].axvline(x=start,c="green", label='Flare Start')
            ax_dict['C'].axvline(x=stop,c="red",label='Flare Stop')
    except IndexError:
        print('no flares!')
    ax_dict['C'].plot(times,fluxes)
    
        
    # add FAP and harmonic notatio

    ax_dict['D'].plot(periods, powers)
    
    ax_dict['D'].set(xlabel='Period (d)', ylabel='Power')#, yscale='log')

    grey_colors = ['lightgray','darkgray','dimgrey']
    for i in range(len(false_alarm_levels)-1,-1,-1): # Plot them in reverse order so the highest confidence label is 
        #confidence_label = str(100*(1-false_alarm_levels[i]))+'% FAP'
        ax_dict['D'].axhline(y=(faps[i]), xmin=1/10, xmax=window_length/3, color = grey_colors[i],lw=1)
    
 
    fap_99 = faps[0]
    more_than_fap_mask = np.where(powers>fap_99)
    periods = periods[more_than_fap_mask]
    powers = powers[more_than_fap_mask]
    sorted_indices = np.argsort(powers)[::-1]
    sorted_powers = powers[sorted_indices][0:20]
    sorted_periods = periods[sorted_indices][0:20]
    if len(sorted_periods)>0: 
        best_period = sorted_periods[0] # get period corresponding to highest power
        best_period_power = sorted_powers[0]
    else: 
        best_period = 0
        best_period_power = 0
    has_harmonic = False 
    if best_period != 0: 
        for i in range(2,6): 
            lower = best_period/i
            upper = best_period*i
            for test_period in sorted_periods: 
                lower_ratio = lower/test_period
                upper_ratio = test_period/upper
                if 1.01 > lower_ratio > 0.99: 
                    has_harmonic = True
                    ax_dict['D'].axvline(x=lower, ymin=0.8, ymax=1, color='indianred', lw=0.75) # replace with red x line midway through plot
                
                
                
                elif 0.99 < upper_ratio < 1.01: 
                    ax_dict['D'].axvline(x=upper, ymin=0.8, ymax=1, color='indianred', lw=0.75)
                    has_harmonic = True
    # sigma clipped light curve 
    ax_dict['E'].scatter(times, fluxes, s=3)
    ax_dict['E'].set(xlabel='Time (d)', ylabel=r'$/sigma$'+'-clipped Normalized Flux')
    e_xlabel = 'Phase (P='+str(round(best_period,2))+' d)'
    if best_period != 0: 
        phased_dates = np.mod(times, best_period)/best_period # phase the dates
        ax_dict['F'].scatter(phased_dates, fluxes, s=3)
        
    else: 
        e_xlabel = 'Phase (P= nan d)'
    ax_dict['F'].set(xlabel=e_xlabel, ylabel='Normalized Flux (FIX)')

    top_periods.append(best_period)
    fap_99s.append(fap_99)
    is_harmonic.append(has_harmonic)
    top_period_powers.append(best_period_power)
    # metadata to save after each iteration: 
    '''

    Note: end results for the entire iteration should be one .csv with id column and columns for the parameters below, which are populated by row/ID basis  

    LombScargle Related: 
    > top_period
    > top_period_power
    > fap_99 
    > top period is harmonic (True/False) # fix because is vs possesses is different 
    
    Flare Related: 
    > add number of flares

    '''


    plt.show()
    #plt.imsave('C:/Users/60002/Documents/GitHub/sunnyhills/data/Mosaic_plots/'+tic_id)
    fig.savefig('C:/Users/60002/Documents/GitHub/sunnyhills/data/New_mosaics/'+tic_id,transparent=False,facecolor="white")
    ## LC MOSAIC KEY FOR NOW 
    """
    A: raw light curve 
    B: flare frequency plot
    C: raw light curve with annotated flares
    D: periodogram with annotated harmonics # detrending is doing weird things
    E: iterative 3 sigma clipped light curve
    F: Phase folded light curve 
    """


In [ ]:
lc_dir = 'C:/Users/60004/Documents/GitHub/sunnyhills/data/LightCurve_keys'
tic_id = '118807628'
make_plot(tic_id, lc_dir)


In [ ]:
tic_ids = pd.read_csv('C:/Users/60002/Documents/GitHub/sunnyhills/data/knownids.csv')
for id in tic_ids['TIC_ID']:
   lc_dir = 'C:/Users/60002/Documents/GitHub/sunnyhills/data/LightCurve_keys'
   id =int(id)
   make_plot(str(id), lc_dir)

In [ ]:
del ids[-1]
df = pd.DataFrame({'ID':ids,'Top_Period':top_periods,'Top_Period_Power':top_period_powers,'Fap_99':fap_99s,'Is_Harmonic':is_harmonic,'Num_Flares':num_flares,'Is_Powerlaw':is_powerlaw})
df.to_csv('C:/Users/60002/Documents/GitHub/sunnyhills/data/mosaic_key.csv')
